In [48]:
!pip install -v --upgrade --force-reinstall --no-build-isolation --no-binary dtaidistance dtaidistance

Using pip 21.3.1 from /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip (python 3.6)
  Using cached dtaidistance-2.3.7.tar.gz (808 kB)
  Running command /home/ec2-user/anaconda3/envs/python3/bin/python /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pip/_vendor/pep517/in_process/_in_process.py prepare_metadata_for_build_wheel /tmp/tmp6t3v9czz
  running dist_info
  creating /tmp/pip-modern-metadata-a1k1knce/dtaidistance.egg-info
  writing /tmp/pip-modern-metadata-a1k1knce/dtaidistance.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-a1k1knce/dtaidistance.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-modern-metadata-a1k1knce/dtaidistance.egg-info/requires.txt
  writing top-level names to /tmp/pip-modern-metadata-a1k1knce/dtaidistance.egg-info/top_level.txt
  writing manifest file '/tmp/pip-modern-metadata-a1k1knce/dtaidistance.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-modern-metadata-a1k1knce/d

In [49]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from collections import defaultdict
from tqdm import tqdm 
from heapq import nlargest
from dtaidistance import dtw

tqdm.pandas()

In [2]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [3]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [4]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:11<00:00, 39660.39it/s]


In [5]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [6]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]
len(career_paths)

113724

In [7]:
career_paths_train = career_paths.iloc[:int(0.8 * 113724)]
career_paths_test = career_paths.iloc[int(0.8 * 113724):]

In [40]:
s = career_paths_train.apply(lambda x : x[-2])
second_to_last_jobs = s.to_frame().groupby("isco_code4").apply(lambda x: set(x.index))

In [ ]:
results = defaultdict(lambda: defaultdict(int))

for c, jobs in tqdm(career_paths_train.iteritems()):
    similar = second_to_last_jobs[jobs[-2]]
    similar = career_paths_train.loc[similar]
    
    jobs = jobs[:-1].astype("double")

    for j in similar.iteritems():
        c2, jobs2 = j

        if c != c2:
            results[c][c2] = dtw.distance(jobs, 
                                          jobs2.astype("double"), 
                                          use_c=True, 
                                          use_pruning=True)

In [9]:
most_common = pd.Series({k: set(nlargest(10, v, key=v.get)) for k, v in switches.items()})

In [10]:
most_common.head()

208    {185, 229, 335, 177, 209, 344, 345, 348, 189, ...
348    {195, 229, 207, 208, 335, 177, 344, 345, 189, ...
344    {323, 195, 185, 332, 335, 208, 177, 117, 343, ...
345    {323, 195, 326, 199, 332, 335, 208, 177, 343, ...
185    {193, 195, 227, 174, 208, 177, 344, 345, 188, ...
dtype: object

In [11]:
most_common_switch = career_paths_test.progress_apply(lambda x: x[-1] in most_common.loc[x[-2]] if x[-2] in most_common.index else False).mean()

100%|██████████| 22745/22745 [00:00<00:00, 46964.59it/s]


In [12]:
most_common_switch, 1.96 * np.sqrt(((1 - most_common_switch)* (most_common_switch)) / (113724*0.2))

(0.6130138491976258, 0.006329914804694123)

In [13]:
print(f"Most common switch accuracy: {most_common_switch * 100:>.2f}%")

Most common switch accuracy: 61.30%
